# Maximum Gift Exchange Chaos

A model designed to explore the impact of different gift exchange rules (e.g. White Elephant, Green Grinch, etc.). The primary goal is to find the rules that lead to maximum chaos, as defined as maximum number of gift steals (possibly -- probably? -- within some time constraint).

## Gift Exchange Model

We can model a gift exchange as a game with an equal number of `G` `gifts` and `P` `players`. Players `value` each gift according to their personal preferences.

    
### Rule Variants
Some gift exchange variants use slightly different rules. These variants can be mixed and matched.

#### Additional end-game swapping
This variant adds extra swaps at the end of the game after `p1` makes their swap. If `p1` does swap with another player's gift, that player becomes active and repeats step 4, and so on until the active player elects not to swap or there are no more eligible gifts.

#### All gifts start unwrapped
In this variant, all gifts in the pool are unwrapped. This means that, instead of players learning their preferences as gifts are opened, all players have complete information about their gift preferences throughout the entire game.

#### Per-round stealing constraints
- A gift can only be stolen `St` times per round, regardless of who is doing the stealing. In other words, once a gift is stolen within a given round, it is ineligble for the remainder of that round.
- No more than `Sg` gifts can be stolen per round. Once `Sg` is reached, the round concludes and a new round begins by the next player becoming active in step 2.

#### Per-player stealing constraints
In this variant, there is also a cap on the number of times `Sp` each *player* can be stolen from. For example, once a given player `Sp_n` has been stolen from `Sp_n = Sp` times, the gift they hold at that point is now ineligible. 

#### No gift stealing constraint
In this variant, there is no cap on the number of times a given gift can be stolen. In other words, `S = infinity`.

### Gift Values
Players have personal preferences and value gifts accordingly. Each `player` has a (potentially) unique subjective `subj_value` for each `gift`. We can therefore model gift subjective value as a `G` by `P` matrix of values `subj_value_gp in range(100)`.

But, if we allow for the notion that some gifts are objectively better than others (almost all gifts will be valued higher than a spray-painted tamborine, for example), we can model `subj_value` as normally distributed variance around a common `obj_value` level. 

In [138]:
import networkx as nx
import numpy as np
import math
import pandas as pd
import scipy.stats as stats

## Inititial Conditions and Setup

### First, we set some initial conditions for our model

In [151]:
# game setup
player_count = 10
gift_count = player_count
gift_value_scale = (0,1)
gift_obj_value_distr = {'mean': .5, 'stdev': .2}
gift_subj_value_stdev = .2

# game rules
gift_steals_max = 3
player_steals_max = None
single_swap = True

# player strategies
steal_threshold = .67

### Next, we create a two classes to model the simulation's state

#### GiftExchangeGift Class

Each **gift** has the following attributes:
- `open`: a boolean marking whether the gift is open (`True`) or wrapped (`False`)
- `owned_by`: the id of the `Player` that currently owns the gift; otherwise `None`
- `steals`: a counter for the number of times the gift is stolen (including swaps)
- `obj_value`: the gift's objective value, set randomly within the `gift_value_scale` based on the `gift_obj_value_distr` parameter
- `eligible`: a boolean marking whether the gift can still be stolen; must be `False` if either `owned == False` or `steals < gift_steals_max`

In [ ]:
class Gift:
    ids = []
    open = []
    owned = []
    eligibles = []
    
    def __init__(self, id, value_scale, value_distr, wrapped):
        self.id = id
        self.open = not wrapped
        self.owned_by = []
        self.steals = 0
        self.is_eligible = True
        self.obj_value = set_obj_value(value_scale, value_distr)
        
        ids.append(id)
        open.append(id) if not wrapped else open
        eligibles.append(id)
    
    # helper function to create a truncated normal distribution
    def create_trunc_norm_distr(scale, mean, stdev):
        lower, upper = scale[0], scale[1]
        return stats.truncnorm(
            (lower - mean) / stdev,
            (upper - mean) / stdev,
            loc = mean,
            scale = stdev)
    
    # set an objective value for a gift
    def set_obj_value(scale, distr):
        mean, stdev = distr['mean'], distr['stdev']
        trunc_norm_distr = create_trunc_norm_distr(scale, mean, stdev)
        return math.floor(trunc_norm_distr.rvs(1))

#### Player Class

Each **player** has the following attributes:
- `brought`: the id of the `Gift` the player brought to the game
- `owns`: the id of the `Gift` the player currently owns; otherwise `None`
- `steals`: a counter for the number of times the player has a gift *stolen from them*
- `stealable`: a boolean marking whether the player can still be stolen *from*
- `active`: a boolean marking whether it's this player's turn to open, steal, or swap a gift
- `subj_values`: Subjective value for a given player&rightarrow;gift is set randomly around the gift's `obj_value` based on the `gift_subj_value_stdev` parameter

In [ ]:
class Player:
    ids = []
    stealables = []
    active = []
    
    def __init__(self, id, value_scale, subj_value_stdev, strategy, gifts):
        self.id = id
        self.strategy = strategy
        self.steals = 0
        self.is_stealable = True
        self.owns = []
        self.is_active = True if id == 0 else False
        self.subj_values = {}
        
        ids.append(id)
        stealables.append(id)
        is_active = id if id == 0 else False
        
        for gift in Gift.ids:
            self.subj_values[gift] = set_gift_subj_value(gift_value_scale, gift.obj_value, gift_subj_value_stdev)
        
        
    # helper function to create a truncated normal distribution
    def create_trunc_norm_distr(scale, mean, stdev):
        lower, upper = scale[0], scale[1]
        return stats.truncnorm(
            (lower - mean) / stdev,
            (upper - mean) / stdev,
            loc = mean,
            scale = stdev)
    
    # set a subjective value for a gift around its objective value
    def set_gift_subj_value(scale, obj_value, stdev):
        distr = create_trunc_norm_distr(scale, obj_value, stdev)
        return math.floor(distr.rvs(1))

## Policies

### Game Rules and Logic

#### Basic Rules
Each player brings one gift to the game and leaves with a (typically) different gift.

The game begins with all gifts wrapped, i.e. each player knows the content of only the gift they brought and none other.

A player order is determined (by some arbitrary method, e.g. random).

1. The first player `p1` selects one of the gifts from the set of unwrapped gifts and opens it.
2. The next player `p2` is now active, and has the option to either:
    1. Open a gift from the pool, leading to a repeat of step 2 with `p3` active
    2. Steal an eligible open gift, leading to a repeat of step 2 with `p1` active
3. Repeat step 2 until all gifts have been opened (i.e. there are no gifts remaining in the pool).
4. `p1` is now active again and has the option to (forcibly) swap with another player for their eligible gift. The game ends once `p1` has swapped or decided to not swap.

A new **round** begins when a new gift is opened from the pool.

##### Stealing
- Each gift can only be stolen up to `S` times, including swaps. A given gift `G_n` is eligible for stealing (or swapping) while `S_n < S`. For example, when `S = 3`, each gift can only be stolen 3 times. After that, the gift is no longer eligible and no player may steal it or swap for it; a player holding an ineligible gift will leave the game with that gift.
- Within a given round, gifts cannot be stolen back by the player who began the turn holding that gift.

    1. If they do not swap, the game ends.
    2. If they do swap with another player, step 4 repeats with that swapped player active

### Player Strategies

1. Steal second most valuable open gift
2. Steal most valuable open gift
3. Steal open gift if `subj_value` is greater than...
    1. Average of `subj_values` of the opened gifts
    2. The maximum `subj_values` of the opened gifts
    3. An arbitrary threshold, e.g. 80
4. Steal an open gift if about to become unstealable

Possible strategies (from https://github.com/BenCasselman/YankeeSwap):
1. Player steals with probability p = (number of gifts taken) / N (naive)
2. Player always steals most valuable gift available
3. Player always steals second-most-valuable gift available (if only one gift is available, player steals that one)
4. Player never steals
5. Player steals if any stealable gift has value (to them) greater than estimated underlying value of average gift
6. Player steals about-to-be unstealable gift if one available greater than estimated underlying value of avg gift
7. Same as #5 but factor in knowledge of gift player brought.
8. Ghosh-Mahdian: Player steals if best available gift has value > theta


To keep things simple at first, we'll use a single strategy for all players: 
- steal the highest `subj_value` open gift if `subj_value` >= 0.67

In [154]:
# helper functions

# get all gift nodes
def get_gift_nodes(network):
    # gifts = extract nodes with type == gift
    # return gifts
    pass

# get IDs for open gifts that are either unowned or stealable
def get_eligible_open_giftIDs():
    # find gifts that are open and eligible and not owned by stealable==False players
    # return gifts
    pass

def get_brought_gift_for(playerID):
    # find gift for which playerID --> gift brought edge == True
    # return gift
    pass

def get_owned_gift_for(playerID):
    # find gift for which playerID --> gift owns edge == True
    # return gift
    pass

def get_eligible_known_gifts_for(playerID):
    
    # brought_gift = get_brought_gift_for(player)
    
    # eligible_open_gifts = get_eligible_open_giftIDs()
    
    # if brought_gift not in eligible_open_gifts:
        # eligible_known_gifts = eligible_open_gifts.append(brought_gift) 
        
    # else: 
        # eligible_known_gifts = eligible_open_gifts
        
    # return eligible_known_gifts
    pass
    
def get_subj_gift_values(playerID, giftIDs):
    # get player subjective gift values for each gift in giftIDs
    # return dictionary with giftIDs and subjective values
    pass
    
def sorted_subj_gift_values(playerID, giftIDs): # returns a list of tuples
    # unsorted = get_subj_gift_values(playerID, giftIDs)
    # sorted = sorted((key, value) for (key, value) in unsorted.items())
    # return sorted
    pass
    
def get_unowned_gifts():
    # find all unowned giftsIDs
    pass

def strategy_steal_if_above_threshold(player, steal_threshold):
    # eligible_known_gift_values = sorted_subj_gift_values(get_eligible_known_gifts_for(player))
    
    # best_eligible_known_gift = eligible_known_gift_values[0][0]
    # best_eligible_known_gift_value = eligible_known_gift_values[0][1]
    
    # if best_eligible_known_gift is in get_unowned_gifts():
        # gift_intention = (open_gift, best_eligible_known_gift)
        
    # elif best_eligible_known_gift_value >= steal_threshold:
        # gift_intention = (steal_gift, best_eligible_known_gift)
        
    # else: #i.e. if best_eligible_known_gift is owned and under steal_threshold
        # random_gift = select an unowned gift at random
        # gift_intention = (open_gift, random_gift)
    
    pass

def strategy_swap_gifts(player):
    # swap owned gift for gift with highest subjective value
    
    # owned_gift = 
    
    # other_gifts = 
    
    # best_other_gift = sorted_subj_gift_values(player, other_gifts)[0][1]
    
    # gift_intention = (swap_gifts, best_other_gift)
    
    pass

## State Update Functions

In [146]:
# player actions

def open_gift(active_player, gift): 
    # if gift is wrapped:
    
        # change gift's open attribute to True

        # change active_player --> gift owns edge to True
        
        # change active_player active attribute to False
        
        # change players.index(active_player + 1) active attribute to True
        
    # else:
        # throw error
    
    pass


def steal_gift(active_player, gift):
    # if gift is open and stealable and owner player is stealable:
        
        # increment owner player steals attribute
        
        # if owner player steals attribute >= player_steals_max:
            # set owner player stealable attribute to False
            
        # change previous owner player --> gift owns edge to False
        
        # change active_player --> gift owns edge to True
        
        # increment gift steals attribute
        
        # if gift steals attribute >= gift_steals_max:
            # set gift stealable attribute to False
            
        # change active_player active attribute to False
            
        # change previous owner player active attribue to True
            
    # else:
        # throw error
        
    pass

def swap_gifts(active_player, target_gift, single_swap):
    # if gift is open and stealable and owner player is stealable:
    
        # increment owner player steals attribute
        
        # if owner player steals attribuet >= player_steals_max:
            # set owner player stealable attribute to False
            
        # change previous owner player --> target_gift owns edge to False
        
        # change active_player --> target_gift owns edge to True
        
        # active_player_gift = active_player's current gift
        
        # change previous owner player --> active_player_gift owns edge to True
        
        # change active_owner --> active_player_gift owns edge to False
        
        # increment gift steals attribute
        
        # if gift steals attribute >= gift_steals_max:
            # set gift stealable attribute to False
            
        # change active_player active attribute to False
            
        # if single_swap = False:
            # change previous owner player active attribute to True
            
    # else:
        # throw error
            
    pass

